Creating a CNN for MNIST using the following architecture:

Image ->  
convolution (32 3x3 filters) -> nonlinearity (ReLU) ->  
convolution (32 3x3 filters) -> nonlinearity (ReLU) -> (2x2 max pool) ->  
convolution (64 3x3 filters) -> nonlinearity (ReLU) ->  
convolution (64 3x3 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> flatten ->
fully connected (256 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax

Model:

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=2)
        self.fc1 = nn.Linear(6400, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        # convolutional layer 1
        x = self.conv1(x)
        x = F.relu(x)
        
        # convolutional layer 2
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # convolutional layer 3
        x = self.conv3(x)
        x = F.relu(x)

        # convolutional layer 4
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        # fully-connected layer 1
        x = x.view(-1, 6400)
        x = self.fc1(x)
        x = F.relu(x)
        
        # fully-connected layer 2
        x = self.fc2(x)
        return x        

Loading data:

In [2]:
import numpy as np
import torch
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

Instantiating the model and defining the loss function, as well as the optimizer:

In [3]:
model = MNIST_CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Training for 3 epochs:

In [4]:
for epoch in trange(3):
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = images 
        y = model(x)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

Accuracy test:

In [5]:
correct = 0
total = len(mnist_test)

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))

  0%|          | 0/100 [00:00<?, ?it/s]

Test accuracy: 0.9916999936103821
